In [1]:
import pandas as pd
import numpy as np

In [2]:
input_csv = "../data/raw/divar_punak_properties_raw.csv"            # Input file path
output_csv = "../data/cleaned/divar_punak_properties_cleaned.csv"   # Output file path

In [3]:
# Read the CSV file
df = pd.read_csv(input_csv)

In [4]:
# Drop duplicates
if 'link' in df.columns:
    df = df.drop_duplicates(subset=['link'])
df = df.drop_duplicates()

In [5]:
# Convert numeric columns
numeric_cols = ['area', 'year_built', 'rooms', 'floor', 'total_price', 'price_per_m2']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [6]:
# Remove rows with critical missing values
df = df.dropna(subset=['area', 'total_price'])

In [7]:
# Remove unrealistic area values
df = df[(df['area'] > 10) & (df['area'] < 1000)]

In [8]:
# Remove unrealistic floor values
if 'floor' in df.columns:
    df = df[(df['floor'].isna()) | ((df['floor'] >= 0) & (df['floor'] <= 50))]

In [9]:
# Standardize price_per_m2
df['price_per_m2'] = df['total_price'] / df['area']

In [10]:
# Remove extreme outliers using IQR on price_per_m2
Q1 = df['price_per_m2'].quantile(0.25)
Q3 = df['price_per_m2'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['price_per_m2'] >= lower_bound) & 
        (df['price_per_m2'] <= upper_bound)]

In [11]:
# Reset index
df = df.reset_index(drop=True)

In [12]:
# Handle missing values for modeling
# Numeric columns: fill with median
numeric_cols_model = ['area', 'year_built', 'rooms', 'floor']
for col in numeric_cols_model:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Categorical columns: fill with most frequent
categorical_cols = ['region', 'has_elevator', 'has_parking', 'has_storage', 'is_new', 'is_renovated', 'has_balcony', 'good_light']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])
        df[col] = df[col].astype(str)

In [13]:
# Save cleaned CSV
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("Cleaning completed.")
print("Final dataset shape:", df.shape)

Cleaning completed.
Final dataset shape: (38, 17)
